In [1]:
from ipynb.fs.full.sentence import Sentence
import math

class MMR(object):
    def __init__(self):
        pass

    
    @staticmethod
    def TF_IDF(Obj_sentences):
        term_freqs = MMR.Term_Freqs(Obj_sentences)
        inverse_dfs = MMR.Inverse_Document_Freqs(Obj_sentences)
        val = {}

        for word in term_freqs:
            tf_inverse_dfs = term_freqs[word] * inverse_dfs[word]

            if val.get(tf_inverse_dfs, None) == None:
                val[tf_inverse_dfs] = [word]
            else:
                val[tf_inverse_dfs].append(word)

        return val

    @staticmethod
    def Inverse_Document_Freqs(Obj_sentences):
        N = len(Obj_sentences)
        inverse_dfs = {}
        words = {}
        w1 = []
        for sent in Obj_sentences:
            for word in sent.get_stemmed_words():
                if sent.getWordFreqencies().get(word, 0) != 0:
                    words[word] = words.get(word, 0) + 1

        for word in words:
            n = words[word]

            try:
                w1.append(n)
                _idf = math.log10(float(N) / n)
            except ZeroDivisentence_ionError:
                _idf = 0

            inverse_dfs[word] = _idf

        return inverse_dfs

    @staticmethod
    def Term_Freqs(Obj_sentences):
        term_freqs = {}

        for sent in Obj_sentences:
            freqs_of_word = sent.getWordFreqencies()

            for word in freqs_of_word.keys():
                if term_freqs.get(word, 0) != 0:
                    term_freqs[word] = term_freqs[word] + freqs_of_word[word]
                else:
                    term_freqs[word] = freqs_of_word[word]
        return term_freqs


    @staticmethod
    def select_best_sentences(Obj_sentences, query, IDF):
        good_sentence = None
        max_ = float("-inf")

        for sent in Obj_sentences:
            sentence_imilarity = MMR.sentence_similarity(sent, query, IDF)

            if sentence_imilarity > max_:
                good_sentence = sent
                max_ = sentence_imilarity
        Obj_sentences.remove(good_sentence)

        return good_sentence



    def summary_make(self, Obj_sentences, good_sentence, query, len_summary, lambdda, IDF):
        summary = [good_sentence]
        current_sum_len = len(good_sentence.get_stemmed_words())

        while (current_sum_len < len_summary):
            MMR__value = {}

            for sent in Obj_sentences:
                MMR__value[sent] = self.get_mmr_score(sent, query, summary, lambdda, IDF)

            max_val = max(MMR__value, key=MMR__value.get)
            summary.append(max_val)
            Obj_sentences.remove(max_val)
            current_sum_len += len(max_val.get_stemmed_words())

        return summary
    
    def get_mmr_score(self, Sentence_i, query, Sentence_j, lambdda, IDF):
        Sentence_immr = MMR.sentence_similarity(Sentence_i, query, IDF)
        left_expr = lambdda * Sentence_immr
        value = [float("-inf")]

        for sent in Sentence_j:
            Sentence_immr_2 = MMR.sentence_similarity(Sentence_i, sent, IDF)
            value.append(Sentence_immr_2)

        right_expr = (1 - lambdda) * max(value)
        SCORE = left_expr - right_expr

        return SCORE
        
    @staticmethod
    def sentence_similarity(u_sent, v_sent, IDF_word):
        upper = 0
        lower = 0

        for word in v_sent.get_stemmed_words():
            upper += u_sent.getWordFreqencies().get(word, 0) * v_sent.getWordFreqencies().get(word, 0) * IDF_word.get(word,
                                                                                                                 0) ** 2

        for word in u_sent.get_stemmed_words():
            lower += (u_sent.getWordFreqencies().get(word, 0) * IDF_word.get(word, 0)) ** 2

        try:
            return upper / math.sqrt(lower)
        except ZeroDivisentence_ionError:
            return float("-inf")

